In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from matplotlib.lines import Line2D
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

# Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_1km_5min.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir+'../../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc', decode_timedelta=True) #***
# res='1km';t_res='5min'
# Np_str='1e6'

# # dx = 1km; Np = 50M; Nz = 95
# #Importing Model Data
# dir2='/home/air673/koa_scratch/'
# data1=xr.open_dataset(dir2+'cm1out_1km_1min_95nz.nc', decode_timedelta=True) #***
# parcel1=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_95nz.nc', decode_timedelta=True) #***
# res='1km'; t_res='1min_95nz'; Np_str='50e6'

# dx = 250m; Np = 50M
#Importing Model Data
dir2='/home/air673/koa_scratch/'
data1=xr.open_dataset(dir2+'cm1out_250m_1min_50M.nc', decode_timedelta=True) #***
parcel1=xr.open_dataset(dir2+'cm1out_pdata_250m_1min_50M.nc', decode_timedelta=True) #***
res='250m'; t_res='1min'; Np_str='50e6'

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions

# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

#####

#Import StatisticalFunctions 
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/Projects/DCI-Project/'
path=dir2+'Functions/'
sys.path.append(path)

import StatisticalFunctions
from StatisticalFunctions import * # import NumericalFunctions 

In [ ]:
#LIMITING Y AXIS
limit_y=True
# limit_y=False

In [ ]:
#NEEDED TO PLOT THE CORRECT DATA #*#*
data_type="Tracked_Properties"

In [ ]:
def limit_axes_to_y(ax, y_min=0, y_max=7, buffer_frac=0.1):
    ax.set_ylim(y_min, y_max)

    x_limited = []
    for line in ax.get_lines():
        xdata, ydata = np.array(line.get_xdata()), np.array(line.get_ydata())
        # Mask for y within limits
        y_mask = (ydata >= y_min) & (ydata <= y_max)
        # Apply mask
        x_visible = xdata[y_mask]
        # Remove NaN or Inf from x_visible
        x_visible = x_visible[np.isfinite(x_visible)]
        x_limited.extend(x_visible)
    if len(x_limited) > 0:
        x_limited = np.array(x_limited)
        x_min, x_max = np.min(x_limited), np.max(x_limited)

        if not (np.isfinite(x_min) and np.isfinite(x_max)):
            print("Warning: Non-finite x-limits detected, skipping set_xlim")
            return
        x_range = x_max - x_min

        if x_range == 0:
            buffer = 0.1  # fixed small buffer
        else:
            buffer = buffer_frac * x_range
        ax.set_xlim(x_min - buffer, x_max + buffer)
    else:
        print("Warning: No visible x data within y limits to set xlim")


In [ ]:
def SavePlot(fig, type1, res, t_res, Np_str, dir2, limit_y=False):
    # Define output directory
    subdir_name = f'{res}_{t_res}_{Np_str}'
    output_dir = os.path.join(dir2, 'Project_Algorithms', 'Tracked_Profiles', 'PLOTS', 'Tracked_Properties', subdir_name)

    if limit_y==True:
        output_dir = os.path.join(output_dir, "SLICE")
    
    # Create the directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    #SAVE PATH
    filename = f'{type1}_Property_Tracked_Profiles_{res}_{t_res}_{Np_str}.jpg'
    save_path = os.path.join(output_dir, filename)
    
    # Save the figure
    fig.savefig(save_path, bbox_inches='tight', dpi=300)
    print(f"Saved figure to: {save_path}")


In [ ]:
def PlotProfile(types, linestyles, fig, gs):
    """
    Plot vertical profiles for multiple cloud types and return axes.

    Parameters
    ----------
    types : list of str
        List of type names (e.g., ["CL", "nonCL", "SBZ"]).
    linestyles : list of str
        Matching list of linestyles for each type.
    fig : matplotlib.figure.Figure
        Figure object to draw into.
    gs : matplotlib.gridspec.GridSpec
        GridSpec object for subplot placement.
    limit_y : bool, optional
        If True, limit y-axis to 0–7 km, else 0–18 km.

    Returns
    -------
    dict
        Mapping from variable name -> axis object.
    """
    lw = 0.8
    vmf_g_ax = None
    axs_dict = {}  # store axes by variable name
    i = 0          # subplot counter

    for type_idx, (type3, linestyle) in enumerate(zip(types, linestyles)):
        for (var, xlabel) in var_units:
            var_upper = var.upper()

            # Select/create axis
            if var == "vmf_c":
                if vmf_g_ax is None:
                    raise ValueError("Cannot plot vmf_c before vmf_g")
                ax = vmf_g_ax
            elif type_idx == 0:
                ax = fig.add_subplot(gs[i])
                axs_dict[var] = ax
                if var == "vmf_g":
                    vmf_g_ax = ax
                i += 1
            else:
                ax = axs_dict[var]

            # Load profiles
            profile_all   = globals()[f"{type3}_ALL_profile_array_{var_upper}"].copy()
            profile_sh    = globals()[f"{type3}_SHALLOW_profile_array_{var_upper}"].copy()
            profile_dp    = globals()[f"{type3}_DEEP_profile_array_{var_upper}"].copy()
            profile_allSE = globals()[f"{type3}_ALL_profile_array_{var_upper}_SE"].copy()
            profile_shSE  = globals()[f"{type3}_SHALLOW_profile_array_{var_upper}_SE"].copy()
            profile_dpSE  = globals()[f"{type3}_DEEP_profile_array_{var_upper}_SE"].copy()

            profiles = [profile_all, profile_sh, profile_dp]
            SEs      = [profile_allSE, profile_shSE, profile_dpSE]
            colors   = ["black", "green", "blue"]
            labels   = ["ALL", "SHALLOW", "DEEP"]

            for profile, se, color, label in zip(profiles, SEs, colors, labels):
                profile = averaged_profiles(profile)

                # unit conversions
                if var in ["qv", "qcqi", "hmc"]:
                    profile[:, 0] *= 1000
                    se[:, 0] *= 1000
                elif var == "MSE":
                    cpd = 1005.7
                    profile[:, 0] /= cpd
                    se[:, 0] /= cpd

                # plotting
                PlotSE(ax, profile, se, color=color, factor=1)
                ax.plot(profile[:, 0], profile[:, 1],
                        color=color, linestyle=linestyle,
                        label=f"{label} ({type3})", lw=lw)

            ax.set_xlabel(xlabel)
            ax.set_ylabel("z (km)")
            ax.grid(True)

            if limit_y:
                ax.set_ylim(0, 7)
            else:
                ax.set_ylim(0, 20)

    return axs_dict


In [ ]:
##################################
#PLOTTING

In [ ]:
def LoadAllCloudBase():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"all_cloudbase_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        all_cloudbase = pickle.load(f)
    return(all_cloudbase)
min_all_cloudbase=np.nanmin(LoadAllCloudBase())
all_cloudbase=min_all_cloudbase
print(f"Minimum Cloudbase is: {min_all_cloudbase}\n")

In [ ]:
def LoadMeanLFC():
    dir2 = dir + f'Project_Algorithms/Tracking_Algorithms/OUTPUT/'
    in_file = dir2 + f"MeanLFC_{res}_{t_res}_{Np_str}.pkl"
    with open(in_file, 'rb') as f:
        MeanLFC = pickle.load(f)
    return MeanLFC
MeanLFC=LoadMeanLFC()
print(f"Mean LFC is: {MeanLFC}\n")

In [ ]:
def averaged_profiles(profile):
    out_var=profile[ (profile[:, 1] > 1)]; #gets rid of rows that have no data\n"
    out_var=np.array([out_var[:, 0] / out_var[:, 1], out_var[:, 2]]).T #divides the data column by the counter column
    return out_var

In [ ]:
# List of variables with their corresponding labels and x-axis titles
var_units = [
    ("qv", r"$q_v$ (g/kg)"),
    ("qcqi", r"$q_c+q_i$ (g/kg)"),
    ("RH_VAPOR", r"$RH_v$ (fraction)"),
    # ("RH_ICE", r"$RH_i$ (fraction)"),

    ("w", 'w (m/s)'),
    ("vmf_g", "VMF (g/kg/s)"), ("vmf_c", "VMF (g/kg/s)"),
    ("hmc", "HMC (g/kg/s)"),
    
    # ("th", r"$\theta$ (K)"),
    ("th_v", r"$\theta_v$ (K)"),
    ("th_e", r"$\theta_e$ (K)"),
    ("MSE", r"$MSE$ (K)"),
    
    # ("buoyancy_cm1", r"$B_{cm1}$ $(m/s^2)$"),
    # ("buoyancy_full", r"$B_{full}$ $(m/s^2)$"),
    # ("buoyancy_full_each_t", r"$B_{full\_each\_time}$ $(m/s^2)$"),
    
]

In [ ]:
def PlotSE(ax, profile, SE_profile, color, factor, switch=1, alpha=0.1, min_value=None):
    lower = profile[:, 0] - factor * SE_profile[:, 0] * switch
    upper = profile[:, 0] + factor * SE_profile[:, 0] * switch
    
    if min_value is not None:
        lower = np.maximum(lower, min_value)
    ax.fill_betweenx(profile[:, -1], lower, upper, color=color, alpha=alpha)
    #Example #PlotSE(ax,profile,SE_profile,color=color,factor=factor, min_value=min_value)

In [ ]:
########################################################################
#CL vs nonCL and SBZ

In [ ]:
type1='CL';type2='nonCL'

for data_type2 in [data_type]:
        
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3+f"{data_type2}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    key_list=[]
    with h5py.File(filePath, 'r') as h5f:
        keys=h5f.keys()
        for key in keys:
            globals()[key] = h5f[key][:]
            if '_squares' not in key:
                key_list.append(key)
        # print(keys)
    
    #CALCULATING STANDARD DEVIATION
    try:
        for key in key_list:
            globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
            # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
            # print(key)
    except:
        #*#*#
        #IF SE NOT AVAILABLE, MAKE DUMMY VARIABLES
        for key in key_list:
            dummy=globals()[key].copy()
            dummy[:,0]=np.zeros(len(dummy[:,0]))
            dummy=dummy[dummy[:,1]>1]
            globals()[key+f"_SE"]=dummy

In [ ]:
type1='SBZ';type2='nonSBZ'

for data_type2 in [data_type]:
        
    dir3=dir+'Project_Algorithms/Tracked_Profiles/OUTPUT_FILES/'
    filePath=dir3+f"{data_type2}_"+f"{type1}_{type2}_tracked_profiles_{res}_{t_res}_{Np_str}.h5"
    key_list=[]
    with h5py.File(filePath, 'r') as h5f:
        for key in h5f.keys():
            globals()[key] = h5f[key][:]
            if '_squares' not in key:
                key_list.append(key)
    
    #CALCULATING STANDARD DEVIATION
    try:
        for key in key_list:
            globals()[key+f"_SE"]=ProfileStandardError(globals()[key],globals()[key+f"_squares"])
            # globals()[key+f"_SE"]=ProfileStandardDeviation(globals()[key],globals()[key+f"_squares"])
            # print(key)
    except:
        #*#*#
        #IF SE NOT AVAILABLE, MAKE DUMMY VARIABLES
        for key in key_list:
            dummy=globals()[key].copy()
            dummy[:,0]=np.zeros(len(dummy[:,0]))
            dummy=dummy[dummy[:,1]>1]
            globals()[key+f"_SE"]=dummy

In [ ]:
type1='CL';type2='nonCL';type3='SBZ'
types=[type1,type2,type3]
linestyles=['solid','dashed','dashdot']

fig = plt.figure(figsize=(15, 12))
gs = gridspec.GridSpec(3, 3, figure=fig)
PlotProfile(types,linestyles,fig,gs)

# FIX TICKS
axes=fig.get_axes()
SnapLimitsToTicks(axes, dim='x')
for axis in axes:
    MatchAxisLimits([axis],dim='x')

# FIXING TICKS
MatchAxisLimits(axes[3:4+1],dim='x')

#ADDING AXVLINES
for ax in axes[3:5+1]:
    ax.axvline(0,color='gray',linestyle='dashed',zorder=-10)

#SCIENTIFIC NOTATION
apply_scientific_notation([axes[5]],decimals=2)
apply_scientific_notation(axes[0:4+1],decimals=2,scientific=False)
apply_scientific_notation(axes[6:],decimals=1,scientific=False)

# Cloud base lines (added after all data plotted)
hline_lw = 1.2
for ax in axes:
    ax.axhline(all_cloudbase, color="purple", linestyle="dashed", lw=hline_lw, zorder=-100)
    ax.axhline(MeanLFC / 1000, color="green", linestyle="dashed", lw=hline_lw, zorder=-100)

# ===== Legend: Colors for cloud depth =====
color_legend_lines = [
    Line2D([0], [0], color='black', linestyle='solid', linewidth=2, label='ALL'),
    Line2D([0], [0], color='green', linestyle='solid', linewidth=2, label='SHALLOW'),
    Line2D([0], [0], color='blue', linestyle='solid', linewidth=2, label='DEEP'),
]

axes[2].legend(  # Use the last axis, or any appropriate index like 0, 1, 2
    handles=color_legend_lines,
    loc='upper left',
    title='Cloud Types',
    title_fontsize=10,
    fontsize=9,
    frameon=True
)

# ===== Legend: Types (linestyles) =====
custom_lines = [
    Line2D([0], [0], color='black', linestyle='solid', linewidth=1.5, label='CL'),
    Line2D([0], [0], color='black', linestyle='dashed', linewidth=1.5, label='nonCL'),
    Line2D([0], [0], color='black', linestyle='dashdot', linewidth=1.5, label='SBF'),
    # No nonSBZ line — it’s not plotted
]

fig.legend(
    handles=custom_lines,
    loc='upper center',
    ncol=4,
    fontsize=10,
    title='Types',
    title_fontsize=12,
    bbox_to_anchor=(0.5, 0.93),
    borderaxespad=0,
    frameon=True
)

#SAVING FIGURE
SavePlot(fig, type1 + type2 +"_"+ "SBF", res, t_res, Np_str, dir2, limit_y)

In [ ]:
limit_y=False

In [ ]:
################################################
#TESTING

In [ ]:
# import matplotlib.pyplot as plt

# # Plot with double x-axis
# fig, ax1 = plt.subplots()

# # First x-axis for W
# test_var = CL_ALL_profile_array_W
# out1 = averaged_profiles(test_var)
# line1, = ax1.plot(out1[:, 0], out1[:, 1], 'k-', label='W (m/s)')
# ax1.set_xlabel('W (m/s)', color='k')
# ax1.set_ylabel('z (km)')
# ax1.tick_params(axis='x', labelcolor='k')

# # Twin x-axis for QI and QCQI
# #1
# ax2 = ax1.twiny()
# test_var = CL_ALL_profile_array_QI
# out2 = averaged_profiles(test_var)
# line2, = ax2.plot(out2[:, 0]*1000, out2[:, 1], 'b-', label='QI (g/kg)')
# #2
# test_var = CL_ALL_profile_array_QCQI
# out3 = averaged_profiles(test_var)
# line3, = ax2.plot(out3[:, 0]*1000, out3[:, 1], 'b--', label='QC+QI (g/kg)')
# ax2.set_xlabel('Q (g/kg)', color='b')
# ax2.tick_params(axis='x', labelcolor='b')
# #3
# test_var = CL_ALL_profile_array_QR
# out3 = averaged_profiles(test_var)
# line3, = ax2.plot(out3[:, 0]*1000, out3[:, 1], 'g-', label='QR (g/kg)')
# ax2.tick_params(axis='x', labelcolor='b')

# # Combine all lines for a single legend
# lines = [line1, line2, line3]
# labels = [line.get_label() for line in lines]

# plt.title('ALL CL Tracked Profiles: W and QI (and QCQI) vs z')
# plt.legend(lines, labels, loc='lower right')  # Adjust loc as needed
# plt.show()


In [ ]:
# import matplotlib.pyplot as plt

# # Plot with double x-axis
# fig, ax1 = plt.subplots()

# # First x-axis for W
# test_var = CL_ALL_profile_array_BUOYANCY
# out1 = averaged_profiles(test_var)
# line1, = ax1.plot(out1[:, 0], out1[:, 1], 'k-', label='W (m/s)')
# ax1.set_xlabel('W (m/s)', color='k')
# ax1.set_ylabel('z (km)')
# ax1.tick_params(axis='x', labelcolor='k')

# # Twin x-axis for QI and QCQI
# ax2 = ax1.twiny()
# test_var = CL_ALL_profile_array_TH_E
# out2 = averaged_profiles(test_var)
# line2, = ax2.plot(out2[:, 0], out2[:, 1], 'b-', label='QI (g/kg)')

# ax2.set_xlabel('TH (K)', color='b')
# ax2.tick_params(axis='x', labelcolor='b')

# # Combine all lines for a single legend
# lines = [line1, line2, line3]
# labels = [line.get_label() for line in lines]

# plt.title('ALL CL Tracked Profiles: BUOYANCY and TH vs z')
# plt.legend(lines, labels, loc='lower center')  # Adjust loc as needed
# plt.show()


In [ ]:
# #OLD PLOT (some glitches)

# def PlotProfile(types,linestyles,fig,gs):
#     lw = 0.8

#     vmf_g_ax = None
#     axs_dict = {}  # Store axes by variable name
#     i = 0  # subplot counter

#     for type_idx, (type3, linestyle) in enumerate(zip(types, linestyles)):
#         profiles = []
#         SE_profiles = []
#         for var, label in var_units:
#             var_upper = var.upper()
#             profiles.append((
#                 globals()[f"{type3}_ALL_profile_array_{var_upper}"].copy(),
#                 globals()[f"{type3}_SHALLOW_profile_array_{var_upper}"].copy(),
#                 globals()[f"{type3}_DEEP_profile_array_{var_upper}"].copy(),
#                 label
#             ))

#             SE_profiles.append((
#                 globals()[f"{type3}_ALL_profile_array_{var_upper}_SE"].copy(),
#                 globals()[f"{type3}_SHALLOW_profile_array_{var_upper}_SE"].copy(),
#                 globals()[f"{type3}_DEEP_profile_array_{var_upper}_SE"].copy(),
#                 None
#             ))

#         for idx, (var, (ALL, SHALLOW, DEEP, xlabel), (ALL_SE, SHALLOW_SE, DEEP_SE, _)) in enumerate(zip(var_units, profiles, SE_profiles)):
#             var_name = var[0]

#             # Axis setup (only once per variable)
#             if var_name == 'vmf_c':
#                 if vmf_g_ax is None:
#                     raise ValueError("Cannot plot vmf_c before vmf_g.")
#                 ax = vmf_g_ax
#             elif type_idx == 0:
#                 ax = fig.add_subplot(gs[i])
#                 axs_dict[var_name] = ax
#                 if var_name == 'vmf_g':
#                     vmf_g_ax = ax
#                 i += 1
#             else:
#                 ax = axs_dict[var_name]

#             profile_list = [ALL, SHALLOW, DEEP]
#             SE_profile_list = [ALL_SE, SHALLOW_SE, DEEP_SE]
#             colors = ['black', 'green', 'blue']
#             labels = ['ALL', 'SHALLOW', 'DEEP']

#             for profile, SE_profile, color, label1 in zip(profile_list, SE_profile_list, colors, labels):
#                 profile = averaged_profiles(profile)

#                 if var_name in ['qv', 'qcqi', 'hmc']:
#                     profile[:, 0] *= 1000
#                     SE_profile[:, 0] *= 1000
#                 if var_name == 'RH_VAPOR':
#                     min_value = 0
#                 elif var_name == 'MSE':
#                     cpd = 1005.7
#                     profile[:, 0] /= cpd
#                     SE_profile[:, 0] /= cpd
#                     min_value = None
#                 elif var_name in ['qv', 'qcqi']:
#                     min_value = 0
#                 else:
#                     min_value = None

#                 PlotSE(ax, profile, SE_profile, color=color, factor=1, min_value=min_value)
#                 ax.plot(profile[:, 0], profile[:, 1],
#                         color=color, linestyle=linestyle,
#                         label=f"{label1} ({type3})", lw=lw)

#             ax.set_xlabel(xlabel)
#             ax.set_ylabel('z (km)')
#             ax.grid(True)

#             # if idx == 2 and type3 == type2:
#             #     ax.legend(loc='upper left', fontsize=9)#,fontsize='small')

#             if limit_y==True:
#                 limit_axes_to_y(ax, y_min=0, y_max=7)
#             elif limit_y==False:
#                 limit_axes_to_y(ax, y_min=0, y_max=18)

#     # Cloud base lines
#     hline_lw = 1.2
#     for axis in fig.get_axes():
#         axis.axhline(all_cloudbase, color='purple', linestyle='dashed', lw=hline_lw, zorder=-100)
#         axis.axhline(MeanLFC / 1000, color='green', linestyle='dashed', lw=hline_lw, zorder=-100)

#     # Format & save
#     # plt.suptitle(f'(ALL, SHALLOW < 4 km, DEEP > 6 km) Tracked Lagrangian {type1} & {type2} (BL-LFC to Exit Cloudy Updraft) Parcels')
#     plt.tight_layout()

#     if len(fig.get_axes()) > 5:
#         apply_scientific_notation([fig.get_axes()[5]])
